In [2]:
import pandas as pd
import numpy as np
from imageio import imread
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import sys

In [3]:
books = pd.read_csv('BX-Books.csv',sep=";",error_bad_lines=False, encoding='latin-1')
rating = pd.read_csv('BX-Book-Ratings.csv',sep=";",error_bad_lines=False, encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/andrejkacera/opt/anaconda3/lib/python3.7/site-packages/IPython/core

In [4]:
books = books[['ISBN', 'Book-Title']]

In [5]:
rating.shape

(1149780, 3)

In [6]:
books.shape

(271360, 2)

In [7]:
books.duplicated(['Book-Title']).sum()

29225

In [8]:
x=rating['User-ID'].value_counts()>200

In [9]:
y = x[x].index

In [10]:
rating=rating[rating['User-ID'].isin(y)]

In [11]:
rating_with_books=rating.merge(books,on='ISBN')

In [12]:
rating_with_books.drop_duplicates(['Book-Title', 'User-ID'], inplace=True)

In [13]:
number_rating=rating_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()
number_rating.rename(columns={'Book-Rating':'number of rating'},inplace=True)
number_rating.head()

,Book-Title,number of rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [14]:
final_ratings=rating_with_books.merge(number_rating,on='Book-Title')
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,number of rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,82
...,...,...,...,...,...
483418,275970,1892145022,0,Here Is New York,1
483419,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,1
483420,275970,3411086211,10,Die Biene.,1
483421,275970,3829021860,0,The Penis Book,1


In [15]:
rating_with_books.shape

(483423, 4)

In [16]:
final_ratings=final_ratings[final_ratings['number of rating']>=50]

In [17]:
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,number of rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,82
...,...,...,...,...,...
233043,255489,0553579983,7,And Then You Die,50
233044,256407,0553579983,0,And Then You Die,50
233045,257204,0553579983,0,And Then You Die,50
233046,261829,0553579983,0,And Then You Die,50


In [18]:
book_pivot=rating_with_books.pivot_table(columns='User-ID',index='Book-Title',values='Book-Rating')

In [19]:
book_pivot.fillna(0,inplace=True)

In [20]:
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Always Have Popsicles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apple Magic (The Collector's series),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Beyond IBM: Leadership Marketing and Finance for the 1990s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ã?Â?lpiraten.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ã?Â?rger mit Produkt X. Roman.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
model=NearestNeighbors()
book_sparse=csr_matrix(book_pivot)

In [22]:
model.fit(book_sparse)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [23]:
model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1))

(array([[0., 0., 0., 0., 0.]]),
 array([[146153,  33713,  76258, 122595,  15517]]))

In [24]:
book_pivot.iloc[237,:].values.reshape(1,-1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 8., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [25]:
def reco(book_name):
    book_id=np.where(book_pivot.index==book_name)[0][0]
    distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1))



    for i in range(len(suggestions)):
        if i==0:
            print("Наиболее близкие к книге",book_name,"это : ")
        if not i:
            print(book_pivot.index[suggestions[i]])

In [26]:
reco('Animal Farm')

Наиболее близкие к книге Animal Farm это : 
Index(['Animal Farm', 'Big Fish: A Novel of Mythic Proportions',
       'Shoeless Joe', 'The Elric Saga PT. II',
       'Miss Smilla's Feeling for Snow'],
      dtype='object', name='Book-Title')


In [27]:
SVD = TruncatedSVD(n_components=12, random_state=0)
matrix = SVD.fit_transform(book_pivot.values.T)

In [28]:
corr = np.corrcoef(matrix)

/Users/andrejkacera/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/andrejkacera/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [29]:
title = book_pivot.index
title_list = list(title)
samia = title_list.index('Animal Farm')
corr_samia  = corr[samia]
list(title[(corr_samia >= 0.9)])

IndexError: index 10064 is out of bounds for axis 0 with size 899